# OEPandas - Advanced Features

This notebook demonstrates advanced OEPandas features including file I/O, design units, data quality filtering, and performance optimization.

In [13]:
import oepandas as oepd
import pandas as pd
from openeye import oechem, oemolprop
from pathlib import Path
import tempfile
import time

## 1. File I/O

### Reading Different File Formats

OEPandas supports reading various molecular file formats with customizable column names:

```python
# SD files - SD data becomes DataFrame columns
df = oepd.read_sdf(
    "molecules.sdf",
    molecule_column="Mol",      # Name for the molecule column (default: "Molecule")
    title_column="Name",         # Name for the title column (default: "Title")
    usecols=["Activity", "MW"],  # Only read specific SD tags
    numeric=["Activity", "MW"]   # Convert these columns to numeric types
)

# OEB binary files (supports .oeb and .oeb.gz)
df = oepd.read_oeb(
    "molecules.oeb.gz",
    molecule_column="Mol",
    title_column="Name"
)

# SMILES files
df = oepd.read_smi(
    "molecules.smi",
    molecule_column="Mol",
    title_column="Name",
    add_smiles=True,  # Also add canonical SMILES column
    add_inchi_key=True  # Also add InChI key column
)

# CSV with SMILES column
df = oepd.read_molecule_csv(
    "data.csv",
    molecule_columns="SMILES"  # Column(s) to convert to molecules
)

# OERecord databases
df = oepd.read_oedb("records.oedb")
```

### Writing Data

Export molecular data to various formats using the `.chem` accessor:

In [3]:
# Create sample data
sample_data = [
    {"SMILES": "CC(=O)Oc1ccccc1C(=O)O", "Name": "Aspirin", "Activity": 7.5},
    {"SMILES": "CC(C)Cc1ccc(cc1)C(C)C(=O)O", "Name": "Ibuprofen", "Activity": 6.8},
    {"SMILES": "CC(=O)Nc1ccc(cc1)O", "Name": "Acetaminophen", "Activity": 5.2},
]
df = pd.DataFrame(sample_data)
df = df.chem.as_molecule("SMILES")

print(f"Created DataFrame with {len(df)} molecules")
df

Created DataFrame with 3 molecules


,SMILES,Name,Activity
0,<oechem.OEMol; proxy of <Swig Object of type '...,Aspirin,7.5
1,<oechem.OEMol; proxy of <Swig Object of type '...,Ibuprofen,6.8
2,<oechem.OEMol; proxy of <Swig Object of type '...,Acetaminophen,5.2


In [4]:
# Write to various formats
with tempfile.TemporaryDirectory() as tmpdir:
    output_dir = Path(tmpdir)
    
    # Write to SDF (columns become SD tags)
    df.chem.to_sdf(
        output_dir / "output.sdf",
        primary_molecule_column="SMILES",
        title_column="Name",
        columns=["Activity"]  # Include these columns as SD tags
    )
    print("Wrote SDF file")
    
    # Write to SMILES file
    df.chem.to_smi(
        output_dir / "output.smi",
        primary_molecule_column="SMILES",
        title_column="Name"
    )
    print("Wrote SMILES file")
    
    # Write to molecule CSV (molecules as SMILES strings)
    df.chem.to_molecule_csv(
        output_dir / "output.csv",
        molecule_format="smiles"  # Can also be "sdf", "oeb", etc.
    )
    print("Wrote molecule CSV file")
    
    # Write to OERecord database
    df.chem.to_oedb(
        output_dir / "output.oedb",
        primary_molecule_column="SMILES"
    )
    print("Wrote OEDB file")
    
    # List created files
    print(f"\nFiles created in {output_dir}:")
    for f in output_dir.iterdir():
        print(f"  - {f.name} ({f.stat().st_size} bytes)")

Wrote SDF file
Wrote SMILES file
Wrote molecule CSV file
Wrote OEDB file

Files created in /var/folders/dk/nb_vhq1s6xggs22mtl34dc2m0000gp/T/tmppo5a2b_4:
  - output.oedb (1092 bytes)
  - output.csv (145 bytes)
  - output.sdf (3902 bytes)
  - output.smi (100 bytes)


## 2. Working with Design Units

Design units are OpenEye's container for protein-ligand complexes:

```python
# Read design unit file
df_du = oepd.read_oedu(
    "complexes.oedu",
    design_unit_column="DU",  # Name for design unit column (default: "Design_Unit")
    title_column="Title"       # Name for title column (default: "Title")
)

# Extract components using .chem accessor
df_du["Ligand"] = df_du.DU.chem.get_ligands()
df_du["Protein"] = df_du.DU.chem.get_proteins()

# Calculate ligand properties
df_du["LigandMW"] = df_du.Ligand.apply(oechem.OECalculateMolecularWeight)

# Calculate protein properties
df_du["NumResidues"] = df_du.Protein.apply(
    lambda mol: sum(1 for _ in oechem.OEGetResidues(mol))
)
```

## 3. Data Quality and Filtering

### Filtering Invalid Molecules

OEPandas provides built-in methods for checking and filtering molecule validity:

In [6]:
# Create data with some invalid molecules
test_data = [
    {"SMILES": "CCO", "Name": "Ethanol"},
    {"SMILES": "invalid", "Name": "Invalid1"},
    {"SMILES": "c1ccccc1", "Name": "Benzene"},
    {"SMILES": "not_a_molecule", "Name": "Invalid2"},
    {"SMILES": "CC(=O)O", "Name": "Acetic Acid"},
]
df_test = pd.DataFrame(test_data)
df_test = df_test.chem.as_molecule("SMILES")

print(f"Created DataFrame with {len(df_test)} entries")

Created DataFrame with 5 entries


In [7]:
# Check validity using .chem.is_valid()
validity = df_test.SMILES.chem.is_valid()

print("Molecule validity check:")
for name, is_valid in zip(df_test.Name, validity):
    status = "Valid" if is_valid else "INVALID"
    print(f"  {name}: {status}")

print(f"\nValid molecules: {validity.sum()}")
print(f"Invalid molecules: {(~validity).sum()}")

Molecule validity check:
  Ethanol: Valid
  Invalid1: INVALID
  Benzene: Valid
  Invalid2: INVALID
  Acetic Acid: Valid

Valid molecules: 3
Invalid molecules: 2


In [8]:
# Filter to keep only valid molecules using .chem.filter_valid()
df_valid = df_test.chem.filter_valid("SMILES")

print(f"Original: {len(df_test)} rows")
print(f"After filtering: {len(df_valid)} rows")
print("\nValid molecules:")
df_valid

Original: 5 rows
After filtering: 3 rows

Valid molecules:


,SMILES,Name
0,<oechem.OEMol; proxy of <Swig Object of type '...,Ethanol
2,<oechem.OEMol; proxy of <Swig Object of type '...,Benzene
4,<oechem.OEMol; proxy of <Swig Object of type '...,Acetic Acid


In [9]:
# You can also filter multiple columns at once
# df_valid = df.chem.filter_valid(["SMILES", "Product"])  # Filter rows where both are valid

# Or use is_valid() for custom filtering
df_with_validity = df_test.copy()
df_with_validity["IsValid"] = df_test.SMILES.chem.is_valid()
df_with_validity

,SMILES,Name,IsValid
0,<oechem.OEMol; proxy of <Swig Object of type '...,Ethanol,True
1,None,Invalid1,False
2,<oechem.OEMol; proxy of <Swig Object of type '...,Benzene,True
3,None,Invalid2,False
4,<oechem.OEMol; proxy of <Swig Object of type '...,Acetic Acid,True


### Property-Based Filtering

In [15]:
# Create a larger dataset for filtering examples
drug_data = [
    {"SMILES": "CC(=O)Oc1ccccc1C(=O)O", "Name": "Aspirin"},
    {"SMILES": "CC(C)Cc1ccc(cc1)C(C)C(=O)O", "Name": "Ibuprofen"},
    {"SMILES": "CC(=O)Nc1ccc(cc1)O", "Name": "Acetaminophen"},
    {"SMILES": "Cn1cnc2c1c(=O)n(c(=O)n2C)C", "Name": "Caffeine"},
    {"SMILES": "CN1c2ccc(cc2C(=NCC1=O)c3ccccc3)Cl", "Name": "Diazepam"},
]
df_drugs = pd.DataFrame(drug_data)
df_drugs = df_drugs.chem.as_molecule("SMILES")

# Calculate properties
df_drugs["MW"] = df_drugs.SMILES.apply(oechem.OECalculateMolecularWeight)
df_drugs["XLogP"] = df_drugs.SMILES.apply(oemolprop.OEGetXLogP)
df_drugs["HBD"] = df_drugs.SMILES.apply(oemolprop.OEGetHBondDonorCount)
df_drugs["HBA"] = df_drugs.SMILES.apply(oemolprop.OEGetHBondAcceptorCount)

df_drugs[["Name", "MW", "XLogP", "HBD", "HBA"]]

,Name,MW,XLogP,HBD,HBA
0,Aspirin,180.15742,0.783,1,2
1,Ibuprofen,206.28082,2.878,1,1
2,Acetaminophen,151.16256,0.432,2,1
3,Caffeine,194.19060,-0.607,0,3
4,Diazepam,284.74022,3.214,0,2


In [17]:
# Apply Lipinski's Rule of Five filter
def passes_ro5(row):
    """Check if molecule passes Lipinski's Rule of Five"""
    return (row["MW"] <= 500 and 
            row["XLogP"] <= 5 and 
            row["HBD"] <= 5 and 
            row["HBA"] <= 10)

df_drugs["PassesRO5"] = df_drugs.apply(passes_ro5, axis=1)

print(f"Molecules passing Lipinski's Rule of Five: {df_drugs.PassesRO5.sum()}")
print(f"Molecules failing: {(~df_drugs.PassesRO5).sum()}")

# Filter to RO5-compliant molecules
df_ro5 = df_drugs[df_drugs.PassesRO5]
df_ro5[["Name", "MW", "XLogP", "HBD", "HBA"]]

Molecules passing Lipinski's Rule of Five: 5
Molecules failing: 0


,Name,MW,XLogP,HBD,HBA
0,Aspirin,180.15742,0.783,1,2
1,Ibuprofen,206.28082,2.878,1,1
2,Acetaminophen,151.16256,0.432,2,1
3,Caffeine,194.19060,-0.607,0,3
4,Diazepam,284.74022,3.214,0,2


## Performance Optimization

### Bulk Operations

In [18]:
# Create a larger dataset for performance testing
large_smiles = [
    "CC(=O)Oc1ccccc1C(=O)O",
    "CC(C)Cc1ccc(cc1)C(C)C(=O)O",
    "CC(=O)Nc1ccc(cc1)O",
    "Cn1cnc2c1c(=O)n(c(=O)n2C)C",
    "c1ccccc1",
] * 200  # 1000 molecules

df_large = pd.DataFrame({"SMILES_str": large_smiles})
df_large = df_large.chem.as_molecule("SMILES_str")

print(f"Created DataFrame with {len(df_large)} molecules")

Created DataFrame with 1000 molecules


In [19]:
# Time bulk operations
start = time.time()
atom_counts = df_large.SMILES_str.apply(lambda mol: mol.NumAtoms() if mol else 0)
elapsed = time.time() - start

print(f"Calculated atom counts for {len(df_large)} molecules")
print(f"Total time: {elapsed:.4f} seconds")
print(f"Average: {elapsed/len(df_large)*1000:.3f} ms per molecule")

Calculated atom counts for 1000 molecules
Total time: 0.0024 seconds
Average: 0.002 ms per molecule


In [20]:
# Using built-in validity check (vectorized)
start = time.time()
validity = df_large.SMILES_str.chem.is_valid()
elapsed = time.time() - start

print(f"Checked validity for {len(df_large)} molecules")
print(f"Total time: {elapsed:.4f} seconds")
print(f"Valid molecules: {validity.sum()}")

Checked validity for 1000 molecules
Total time: 0.0016 seconds
Valid molecules: 1000


### Memory-Efficient Batch Processing

In [21]:
def process_in_batches(df, column, func, batch_size=100):
    """Process molecules in batches for memory efficiency"""
    results = []
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        batch_results = batch[column].apply(func)
        results.extend(batch_results)
    
    return pd.Series(results, index=df.index)

# Example: Calculate molecular weights in batches
start = time.time()
mw_values = process_in_batches(
    df_large, 
    "SMILES_str", 
    lambda mol: oechem.OECalculateMolecularWeight(mol) if mol else 0,
    batch_size=100
)
elapsed = time.time() - start

print(f"Processed {len(mw_values)} molecules in batches")
print(f"Time: {elapsed:.4f} seconds")
print(f"Average MW: {mw_values.mean():.2f}")

Processed 1000 molecules in batches
Time: 0.0099 seconds
Average MW: 161.98


## Advanced Molecular Operations

### Conformer Generation

In [22]:
from openeye import oeomega

def generate_conformers(mol, max_confs=10):
    """Generate 3D conformers for a molecule"""
    if mol is None:
        return None
    
    omega = oeomega.OEOmega()
    omega.SetMaxConfs(max_confs)
    omega.SetIncludeInput(False)
    omega.SetStrictStereo(False)
    
    mol_copy = mol.CreateCopy()
    if omega(mol_copy):
        return mol_copy
    return None

# Generate conformers for first molecule
sample_mol = df_drugs.SMILES.iloc[0]
mol_with_confs = generate_conformers(sample_mol, max_confs=5)

if mol_with_confs:
    print(f"Original molecule: {df_drugs.Name.iloc[0]}")
    print(f"Generated {mol_with_confs.NumConfs()} conformers")

Original molecule: Aspirin
Generated 5 conformers


### Molecular Fingerprints for Similarity

In [23]:
from openeye import oegraphsim

def calc_fingerprint(mol):
    """Calculate circular fingerprint"""
    if mol is None:
        return None
    fp = oegraphsim.OEFingerPrint()
    oegraphsim.OEMakeFP(fp, mol, oegraphsim.OEFPType_Circular)
    return fp

def calc_tanimoto(fp1, fp2):
    """Calculate Tanimoto similarity between fingerprints"""
    if fp1 is None or fp2 is None:
        return 0.0
    return oegraphsim.OETanimoto(fp1, fp2)

# Calculate fingerprints
df_drugs["Fingerprint"] = df_drugs.SMILES.apply(calc_fingerprint)

# Calculate pairwise similarity to first molecule
reference_fp = df_drugs.Fingerprint.iloc[0]
df_drugs["Similarity"] = df_drugs.Fingerprint.apply(
    lambda fp: calc_tanimoto(reference_fp, fp)
)

print(f"Similarity to {df_drugs.Name.iloc[0]}:")
df_drugs[["Name", "Similarity"]].sort_values("Similarity", ascending=False)

Similarity to Aspirin:


,Name,Similarity
0,Aspirin,1.000000
2,Acetaminophen,0.183673
1,Ibuprofen,0.147541
4,Diazepam,0.120482
3,Caffeine,0.064516


## Integration with Machine Learning

Prepare molecular descriptors for ML workflows:

In [29]:
# Calculate comprehensive descriptors
df_ml = df_drugs.copy()

# Helper to safely get ring system count (handles tuple return in newer OpenEye versions)
def get_ring_count(mol):
   if mol is None:
       return 0
   result = oechem.OEDetermineRingSystems(mol)
   return result[0] if isinstance(result, tuple) else result

# Physicochemical properties
df_ml["TPSA"] = df_ml.SMILES.apply(lambda mol: oemolprop.OEGet2dPSA(mol) if mol else 0)
df_ml["RotBonds"] = df_ml.SMILES.apply(
   lambda mol: oechem.OECount(mol, oechem.OEIsRotor()) if mol else 0
)
df_ml["RingCount"] = df_ml.SMILES.apply(get_ring_count)
df_ml["HeavyAtoms"] = df_ml.SMILES.apply(
   lambda mol: oechem.OECount(mol, oechem.OEIsHeavy()) if mol else 0
)

# Define feature columns
feature_columns = ["MW", "XLogP", "HBD", "HBA", "TPSA", "RotBonds", "RingCount", "HeavyAtoms"]

# Create feature matrix
X = df_ml[feature_columns].values

print(f"Feature matrix shape: {X.shape}")
print(f"\nFeature statistics:")
df_ml[feature_columns].describe().round(2)


Feature matrix shape: (5, 8)

Feature statistics:


,MW,XLogP,HBD,HBA,TPSA,RotBonds,RingCount,HeavyAtoms
count,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
mean,203.31,1.34,0.80,1.80,48.94,2.00,1.20,14.60
std,49.95,1.64,0.84,0.84,13.97,1.58,0.45,3.36
min,151.16,-0.61,0.00,1.00,32.67,0.00,1.00,11.00
25%,180.16,0.43,0.00,1.00,37.30,1.00,1.00,13.00
50%,194.19,0.78,1.00,2.00,49.33,2.00,1.00,14.00
75%,206.28,2.88,1.00,2.00,61.82,3.00,1.00,15.00
max,284.74,3.21,2.00,3.00,63.60,4.00,2.00,20.00


In [30]:
# Example: Simple clustering based on descriptors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Cluster molecules
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_ml["Cluster"] = kmeans.fit_predict(X_scaled)

print("Clustering results:")
df_ml[["Name", "Cluster"]].sort_values("Cluster")

Clustering results:


,Name,Cluster
0,Aspirin,0
1,Ibuprofen,0
2,Acetaminophen,0
3,Caffeine,0
4,Diazepam,1
